In [5]:
import pickle
import os
import numpy as np
from recal import iso_recal
import torch
from data.fetch_data import get_uci_data
from utils.misc_utils import set_seeds, test_uq
from argparse import Namespace

In [9]:
#for p in ('0.0', '0.25', '0.5', '0.75', '1.0'):
p = '0.0'
gdp = 1
for dataset in ('boston', 'yacht', 'energy', 'concrete',  'kin8nm', 'wine', 'power', 'naval', 'protein'):
    data = []
    for seed in range(5):
        with open(f'cali_run/{dataset}_lossscaled_batch_cal_penalty_pen{p}_wideonly_ens1_bootFalse_gdp{gdp}_seed{seed}.pkl', 'rb') as f:
            d = pickle.load(f)
        
        model_ens = d['model']
        recal_model = iso_recal(d['va_exp_props'], d['va_obs_props'])
        recal_exp_props = torch.linspace(0.01, 0.99, 99)
        
        set_seeds(seed)

        # Fetching data
        data_args = Namespace(data_dir='data/UCI_Datasets', data=dataset, seed=seed)

        data_out = get_uci_data(data_args)

        x_tr, x_va, x_te, y_tr, y_va, y_te, y_al = (
            data_out.x_tr,
            data_out.x_va,
            data_out.x_te,
            data_out.y_tr,
            data_out.y_va,
            data_out.y_te,
            data_out.y_al,
        )
        y_range = (y_al.max() - y_al.min()).item()
        
        (
            recal_te_cali_score,
            recal_te_sharp_score,
            recal_te_obs_props,
            recal_te_q_preds,
            recal_te_g_cali_scores,
            recal_te_scoring_rules
        ) = test_uq(
            model_ens,
            x_te,
            y_te,
            recal_exp_props,
            y_range,
            recal_model=recal_model,
            recal_type="sklearn",
            test_group_cal=True,
        )
        if d['recal_te_cali_score'] is not None:
            print(dataset, seed, d['recal_te_cali_score'])
        if d['recal_te_sharp_score'] is not None:
            print(dataset, seed, d['recal_te_sharp_score']) 
        d['recal_te_cali_score'] = recal_te_cali_score
        d['recal_te_sharp_score'] = recal_te_sharp_score
        data.append(d)
    ece = []
    sharp = []
    ece_recal = []
    sharp_recal = []
    for d in data:
        ece.append(d['te_cali_score'])
        sharp.append(d['te_sharp_score'])
        ece_recal.append(d['recal_te_cali_score'])
        sharp_recal.append(d['recal_te_sharp_score'])
    print(dataset, np.mean(ece), np.mean(sharp), np.std(ece), np.std(sharp))
    print(dataset, np.mean(ece_recal), np.mean(sharp_recal), np.std(ece_recal), np.std(sharp_recal))

Spanning group size from 0.01 to 1.0 in 10 increments


100%|██████████| 10/10 [00:00<00:00, 22.61it/s]


Spanning group size from 0.01 to 1.0 in 10 increments


100%|██████████| 10/10 [00:00<00:00, 22.48it/s]


Obs props not ideal: from 0.0 to 0.9780219793319702
Spanning group size from 0.01 to 1.0 in 10 increments


...

Spanning group size from 0.01 to 1.0 in 10 increments


100%|██████████| 10/10 [00:01<00:00,  7.16it/s]


Spanning group size from 0.01 to 1.0 in 10 increments


100%|██████████| 10/10 [00:01<00:00,  7.19it/s]

protein 0.014511549845337868 0.5398490503609525 0.003594750575321801 0.027279903320313527
protein 0.006941197719424963 0.696608871651297 0.002423151901455195 0.0273165657900291
